# [1] Scientific computation

There are several packages that provide multidimensional data manipulation, optimization, regression, interpolation and visualization, among other possibilities.

## 0. Some arithmetic insights

### Floats

Python uses (hardware) 754 double precision for floats. This means that some floats can be only represented approximately.

In [ ]:
format(0.1, '.20f')

For "infinite" precision float arithmetic you can use [decimal](https://docs.python.org/3/library/decimal.html#module-decimal).

In [ ]:
from decimal import Decimal
format(Decimal(1)/Decimal(10), '.20f')

Lets compute the $\pi$ number using the [Bailey–Borwein–Plouffe formula](https://en.wikipedia.org/wiki/Bailey%E2%80%93Borwein%E2%80%93Plouffe_formula):

$$
\pi = \sum_{k = 0}^{\infty}\left[ \frac{1}{16^k} \left( \frac{4}{8k + 1} - \frac{2}{8k + 4} - \frac{1}{8k + 5} - \frac{1}{8k + 6} \right) \right]
$$

In [ ]:
# https://stackoverflow.com/questions/28284996/python-pi-calculation
from decimal import Decimal, getcontext
getcontext().prec=100
my_pi= sum(1/Decimal(16)**k * 
          (Decimal(4)/(8*k+1) - 
           Decimal(2)/(8*k+4) -
           Decimal(1)/(8*k+5) -
           Decimal(1)/(8*k+6)) for k in range(100))
'{:.100f}'.format(my_pi)

You can visit [100,000 Digits of Pi](http://www.geom.uiuc.edu/~huberty/math5337/groupe/digits.html) to check the correctness this code.

### Integers

In python, integers have arbitrary precision and therefore we can represent an arbitrarily large range of integers (only limited by the available memory).

## 1. SciPy.org's [Numpy](http://www.numpy.org/)

Numpy provides a high-performance multidimensional array object.

### 1.1. Installation

```
pip install numpy
```

### 1.2. Why numpy?

Better running times:

In [ ]:
import numpy as np

In [ ]:
l = list(range(0,100000)); print(type(l), l[:10])
a = np.arange(0, 100000); print(type(a), a[:10])

In [ ]:
%timeit sum(l)

In [ ]:
%timeit np.sum(a)

In [ ]:
!cat sum_array.c
!gcc -O3 sum_array.c -o sum_array
%timeit !./sum_array

Looking for something:

In [ ]:
np.lookfor('invert')

You can also use the tabulator to extend some command of use a wildcard in Ipython:

In [ ]:
np.*?

### 1.3. Creating arrays
A numpy array is a grid of values, all of the same type, indexed by a tuple of nonnegative integers.

### 1.3.1. 1D

Creation:

In [ ]:
a = np.array([1, 2, 3])  # Create a rank 1 array
print(type(a))

In [ ]:
print(a.ndim)

In [ ]:
print(a)

In [ ]:
print(a.shape)

In [ ]:
print(len(a))

In [ ]:
np.linspace(1., 4., 6)

In [ ]:
!cat data.txt
np.genfromtxt('data.txt')

### Indexing:

In [ ]:
print(a[0], a[1])

In [ ]:
a[0] = 0
print(a)

Arrays can be creted from different types of contaniers (which store complex numbers in this case):

In [ ]:
x = np.array([[1,1.0],(1+1j,.3)])
x

### 1.3.2. 2D

With 2 arrays:

In [ ]:
b = np.array([[1,2,3],[4,5,6]])   # Create a rank 2 array
print(b)
print(b.shape)
print(b[1, 1])

With zeroes:

In [ ]:
a = np.zeros((5,5)) # The default dtype is float64
print(a)

With ones:

In [ ]:
a = np.ones((5,5))
print(a)

With an arbitrary scalar:

In [ ]:
a = np.full((5,5), 2)
print(a)

The identity matrix:

In [ ]:
a = np.eye(5)
print(a)

With ramdom data:

In [ ]:
a = np.random.random((5,5))
print(a)

In [ ]:
a = np.random.random((5,5))
print(a)

Filled with zeroes and with a previously defined shape:

In [ ]:
b = np.empty_like(a)
print(b)

With a 1D list comprehension:

In [ ]:
a = np.array([i for i in range(5)])
print(a, a[1], a.shape)

With a 2D list comprehension:

In [ ]:
b = np.array([[j for j in range(10)] for i in range(1)])
print(b, b.shape)

A different 2D list comprehension:

In [ ]:
a = np.array([[i*5+j for j in range(5)] for i in range(5)])
print(a)

Getting elements of a matrix:

In [ ]:
print(a[0,3], a[1,2], a[2,1])

Getting elements of a matrix using "integer array indexing":

In [ ]:
print(a)
print(a[[0, 1, 2], [3, 2, 1]])

The same integer array indexing using comprehension lists:

In [ ]:
print(a[np.array([i for i in range(3)]), np.array([i for i in range(3,0,-1)])])

The same using the `np.arange()` function:

In [ ]:
print(np.arange(3))
print(np.arange(3,0,-1))
print(a[np.arange(3), np.arange(3,0,-1)])

### 1.4. Slicing

In [ ]:
print(a,a[0:],a[1:])

Getting a top-left submatrix:

In [ ]:
print(a[:2,:2])

Getting a bottom-right submatrix:

In [ ]:
print(a[2:,2:])

Getting a column of a matrix:

In [ ]:
print("column 2 =", a[:,2])

Sampling:

In [ ]:
print(a[2::2,::2])

 ### 1.5. Boolean array indexing

Finding the elements bigger than ...

In [ ]:
bool_idx = (a>12)
print(bool_idx)

In [ ]:
print(a[bool_idx])

### 1.6. Elementwise (vectorial-vectorial and vectorial-scalar) math

In [ ]:
a = np.zeros((5,5), np.int32)
print(a)

In [ ]:
a[1:4,1:4] = 1
print(a)

Vectorial-scalar addition:

In [ ]:
a[1:4, 1:4] += 1
print(a)

In [ ]:
b = np.ones((5,5), np.int32)
print(b)

Vectorial addition:

In [ ]:
c = a + b
print(c)

Vectorial substraction:

In [ ]:
d = c - b
print(d)

Vectorial multiplication (not matrix multiplication!):

In [ ]:
c = c * d
print(c)

Floating-point vectorial division:

In [ ]:
c = c / b
print(c)

Fixed-point (integer) vectorial division:

In [ ]:
c = d // b
print(c)

### Broadcasting
In vectorized operations, NumPy "extends" scalars and arrays with one of its dimensions equal to 1 to the size of the other(s) array(s).

In [ ]:
a = np.ones((5,3))
a

In [ ]:
b = np.arange(1) + 1
b

In [ ]:
a+b # 'a' is 5x3 and 'b' is 1x1

In [ ]:
b = np.arange(3)
b

In [ ]:
a+b # 'a' is 5x3 and 'b' is '1x3'

In [ ]:
b = np.arange(5)
b

In [ ]:
b = b.reshape((5,1)) # (Rows, Columns)
b

In [ ]:
a+b

If the arrays have different shapes and s can not be "broadcasted", `ValueError: frames are not aligned` is thrown.

In [ ]:
b = np.arange(4)[:, None]
b

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
a+b

### 1.7. Matricial math
Provides basic matrix computation.

A chessboard matrix:

In [ ]:
a = np.array([[(i+j)%2 for j in range(10)] for i in range(10)])
print(a, a.shape)

... and a 1-column matrix:

In [ ]:
b = np.array([[1] for i in range(10)])
print(b, b.shape)

Product matrix-vector:

In [ ]:
c = np.dot(a,b)
print(c)

Sum of all elements of a matrix:

In [ ]:
print(np.sum(c))

In [ ]:
print(np.sum(a))

Compute the maximum of a matrix:

In [ ]:
print(np.max(c))

Matrix transpose:

In [ ]:
print(c.T, c.T.shape, c.shape)

### How fast is array math?

In [ ]:
a = np.array([[(i+j) for j in range(10)] for i in range(10)])
print(a, a.shape)

In [ ]:
a[:1]

In [ ]:
a[:1].shape

In [ ]:
a[:1][0]

In [ ]:
a[:1][0].shape

In [ ]:
b = a[:1][0]
print(b, b.shape)

Add `b[]` to all the rows of `a[][]` using scalar arithmetic:

In [ ]:
c = np.empty_like(a)
def add():
    for i in range(a.shape[1]):
        for j in range(a.shape[0]):
            c[i, j] = a[i, j] + b[j]
%timeit add()
print(c)

Add b[] to all the rows of a[][] using vectorial arithmetic:

In [ ]:
c = np.empty_like(a)
def add():
    for i in range(a.shape[1]):
        c[i, :] = a[i, :] + b
%timeit add()
print(c)

Add b[] to all the rows of a[][] using fully scalar arithmetic:

In [ ]:
%timeit c = a + b # <- broadcasting is faster
print(c)

### Structured arrays

In [ ]:
x = np.array([(1,2.,'Hello'), (3,4.,"World")],
             dtype=[('first', 'i4'),('second', 'f4'), ('third', 'S10')])
x

In [ ]:
x['first']

In [ ]:
x['second']

In [ ]:
x['third']

## 2. [Matplotlib](http://matplotlib.org)
A Python 2D plotting library.

### 2.1. Installation

```
pip install matplotlib
```

### 2.2. Configure Matplotlib in-line of Ipython (Jupyter) notebook

In [ ]:
%matplotlib inline

### 2.3. Importing it

In [ ]:
import matplotlib.pyplot as plt

### 2.4. Drawing data structures (matrices):

In [ ]:
chess_board = np.zeros([8, 8], dtype=int)
chess_board[0::2, 1::2] = 1
chess_board[1::2, 0::2] = 1
plt.matshow(chess_board, cmap=plt.cm.gray)

### 2.5. Drawing 2D curves

In [ ]:
resolution = 100
x = np.arange(0, 3*np.pi, np.pi/resolution)
si = np.sin(x)
co = np.cos(x)
plt.plot(x, si, c = 'r')
plt.plot(x, co, c = 'g')
plt.legend(['$\sin(x)$', '$\cos(x)$'])
plt.xlabel('radians')
plt.title('sine($x$) vs. cosine($x$)')
plt.xticks(x*resolution, ['0', '$\pi$', '$2\pi$'], rotation='horizontal')
plt.xlim(0,3*np.pi)
plt.show()

### 2.6. Drawing 3D curves

In [ ]:
x = np.array([[(x+y)/25 for x in range(256)] for y in range(256)])
si = np.sin(x)
plt.imshow(si, cmap='hot', interpolation='nearest')
plt.show()

In [ ]:
# https://github.com/AeroPython/Taller-Aeropython-PyConEs16
def funcion(x,y):
    return np.cos(x) + np.sin(y)

x_1d = np.linspace(0, 5, 100)
y_1d = np.linspace(-2, 4, 100)
X, Y = np.meshgrid(x_1d, y_1d)
Z = funcion(X,Y)
plt.contourf(X, Y, Z, np.linspace(-2, 2, 100),cmap=plt.cm.Spectral)
plt.colorbar()
cs = plt.contour(X, Y, Z, np.linspace(-2, 2, 9), colors='k')
plt.clabel(cs)

## 3. [SciPy](https://docs.scipy.org/doc/scipy/reference/)
[SciPy](http://cs231n.github.io/python-numpy-tutorial/#numpy-array-indexing) provides a large number of functions that operate on numpy arrays and are useful for different types of scientific and engineering applications such as:
1. [Custering](https://docs.scipy.org/doc/scipy/reference/cluster.html).
2. [Discrete Fourier Analysis](https://docs.scipy.org/doc/scipy/reference/fftpack.html).
3. [Interpolation](https://docs.scipy.org/doc/scipy/reference/interpolate.html).
4. [Linear algebra](https://docs.scipy.org/doc/scipy/reference/linalg.html).
5. [Signal](https://docs.scipy.org/doc/scipy/reference/signal.html) and [Image processing](https://docs.scipy.org/doc/scipy/reference/ndimage.html).
6. [Optimization](https://docs.scipy.org/doc/scipy/reference/optimize.html).
7. [Sparse matrix](https://docs.scipy.org/doc/scipy/reference/sparse.html) and [sparse linear algebra](https://docs.scipy.org/doc/scipy/reference/sparse.linalg.html).



### 3.1. Installation

```
pip install scipy
```

### 3.1.1. Optimization example

In [ ]:
# http://www.scipy-lectures.org/advanced/mathematical_optimization/
from scipy import optimize

def f(x):
    return -np.exp(-(x - .7)**2)

In [ ]:
sol = optimize.brent(f)
print('min =', sol, '\nx =', f(sol))

In [ ]:
x = np.arange(-10, 10, 0.1)
plt.plot(x, f(x))
plt.plot([sol],[f(sol)], 'ro')
plt.show()

## 4. [Pandas](http://pandas.pydata.org/)
High-performance data structures and data analysis tools for the Python programming language (similar to [R](https://en.wikipedia.org/wiki/R_(programming_language)). Some tools are:
1. [Statistical functions (covariance, correlation)](http://pandas.pydata.org/pandas-docs/stable/computation.html#statistical-functions).
2. [Window functions](http://pandas.pydata.org/pandas-docs/stable/computation.html#window-functions).
3. [Time series](http://pandas.pydata.org/pandas-docs/stable/timeseries.html).
4. [Analysis of sparse data](http://pandas.pydata.org/pandas-docs/stable/sparse.html).

### 4.1. Installation

```
pip3 install pandas
```

### 4.2. Example

Create a table with data:

In [ ]:
import numpy as np
import pandas as pd
df = pd.DataFrame({'int_col' : [1, 2, 6, 8, -1],
                    'float_col' : [0.1, 0.2, 0.2, 10.1, None],
                    'str_col' : ['a', 'b', None, 'c', 'a']})
print(df)
df

Arithmetic average of a column:

In [ ]:
df2 = df.copy()
mean = df2['float_col'].mean()
mean

Replace undefined elements:

In [ ]:
df3 = df['float_col'].fillna(mean)
df3

Create a table by means of columns:

In [ ]:
df4 = pd.concat([df3, df['int_col'], df['str_col']], axis=1)
df4

## 5. [SymPy](http://www.sympy.org/en/index.html)
A Python library for symbolic mathematics. Among others things, it provides:
1. [Symbolic simplification](http://docs.sympy.org/latest/tutorial/simplification.html).
2. [Calculus (derivatives, integrals, limits, and series expansions)](http://docs.sympy.org/latest/tutorial/calculus.html).
3. [Algebraic solver](http://docs.sympy.org/latest/tutorial/solvers.html).
4. [Matrix operations](http://docs.sympy.org/latest/tutorial/matrices.html).
5. [Combinatorics](http://docs.sympy.org/latest/modules/combinatorics/index.html)
6. [Cryptography](http://docs.sympy.org/latest/modules/crypto.html).

### 5.1. Install
```
pip install sympy
```

### 5.2. Example

In [ ]:
from sympy import init_session
init_session(use_latex='matplotlib')

In [ ]:
# https://github.com/AeroPython/Taller-Aeropython-PyConEs16
expr = cos(x)**2 + sin(x)**2
expr

In [ ]:
simplify(expr)

In [ ]:
expr.subs(x, y**2)

In [ ]:
expr = (x + y) ** 2
expr

In [ ]:
expr = expr.expand()
expr

In [ ]:
expr = expr.factor()
expr

In [ ]:
expr = expr.integrate(x)
expr

In [ ]:
expr = expr.diff(x)
expr